In [ ]:
# Husayn El Sharif
comment = """
Use environment: env003 which has tensorflow support
use efficientnet from tensorhub and force 456 x 456 size
"""

In [ ]:
# TensorFlow Import and GPU Check

import tensorflow as tf
import tf_keras as keras # Import tf_keras for Keras 3 compatibility
import tensorflow_hub as hub
print("TF Version:", tf.__version__)
print("TF Hub Version:", hub.__version__)

# Check if GPU is savailable
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs available:")
    for gpu in gpus:
        print(f"- {gpu}")
else:
    print("No GPUs found.")

In [ ]:
# other imports
import numpy as np
import pandas as pd
import os
import glob

import datetime


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report


from IPython.display import Image

import matplotlib.pyplot as plt


import plotly.graph_objects as go
from plotly.subplots import make_subplots

import datetime

from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report



In [ ]:
# Use filenames and folders to get filepath and labels for all  images
all_imagefiles_list = glob.glob(
    'images/**/*.jpg', 
    recursive=True
)  # resized to 456x456 .jpg earlier

labels_temp = [filepath.split('/')[-2] for filepath in all_imagefiles_list]
patient_id = [f"{Path(filepath).stem.split('_')[0]}_{Path(filepath).parent.name}" for filepath in all_imagefiles_list]

# create dataframe with image_filepath, label
full_data_df = pd.DataFrame({
    'image_filepath': all_imagefiles_list,
    'label': labels_temp,
    'patient_id': patient_id
})

full_data_df


In [ ]:
# Split the full data into train, validation, and test sets based on patient_id. 
# Use 80% for training, 10% for validation, and 10% for testing.
# Split by patient_id ensures that images from the same patient do not appear in both training and validation sets or
# training and test sets, or validation and tests sets.
# As eye disease may affect both eyes, this is important to prevent data leakage.

train_ids, not_train_ids = train_test_split(
    full_data_df['patient_id'].unique(), 
    test_size=0.2, 
    random_state=42
)

val_ids, test_ids = train_test_split(
    not_train_ids, 
    test_size=0.5, 
    random_state=42
)

In [ ]:
# create dataframes for train, validation, and test sets

train_df = full_data_df[full_data_df['patient_id'].isin(train_ids)].reset_index(drop=True)
val_df = full_data_df[full_data_df['patient_id'].isin(val_ids)].reset_index(drop=True)
test_df = full_data_df[full_data_df['patient_id'].isin(test_ids)].reset_index(drop=True)

In [ ]:
# how many images of each label (in training data)?
print(train_df["label"].value_counts())
comment = """
Classes appear roughly balanced in quantity.
"""

In [ ]:
# how many images of each label (in validation data)?
print(val_df["label"].value_counts())
comment = """
Classes are reasonably balanced in quantity .
"""

In [ ]:
# how many images of each label (in test data)?
print(test_df["label"].value_counts())
comment = """
Classes are reasonablybalanced in quantity .
"""

In [ ]:
# Create integer class labels
# Instead of one-hot booleans, we’ll use integer class indices (better for CrossEntropyLoss).

full_data_labels = full_data_df['label'].values.tolist()
unique_labels = sorted(np.unique(full_data_labels))  # e.g. ['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']

label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}
idx_to_label = {idx: label for label, idx in label_to_idx.items()}

In [ ]:
NUM_CLASSES = len(unique_labels)

In [ ]:
# X_train and y_train 
train_labels = train_df['label'].values.tolist()

int_labels = [label_to_idx[label] for label in train_labels]

X_train = train_df["image_filepath"].tolist()
y_train = int_labels

In [ ]:
# X_val and y_val 
val_labels = val_df['label'].values.tolist()

int_labels = [label_to_idx[label] for label in val_labels]

X_val = val_df["image_filepath"].tolist()
y_val = int_labels

In [ ]:
# X_test and y_test 
test_labels = test_df['label'].values.tolist()

int_labels = [label_to_idx[label] for label in test_labels]

X_test = test_df["image_filepath"].tolist()
y_test = int_labels

In [ ]:
# check lengths of split data
len(X_train), len(y_train), len(X_val), len(y_val), len(X_test), len(y_test)


In [ ]:
# parameter: image size (assuming square image size)

# parameter: image size 
IMG_SIZE_HEIGHT = 456 # recommended image size for EfficientNetB5
IMG_SIZE_WIDTH = 456
BATCH_SIZE = 32

In [ ]:
# turn data into Tensors
# labels are already in numeric format
# so we need to process the images

# preprocessing images: turning images into tensors
# create helpful functions for pre-processing

comment = '''

1. take an image filepath as input
2. use TF to read the file and savite it to a variable, "image"
3. turn our "image" into a tensors
4. Normalize range of image channels from 0-255 to 0 - 1 (normalization)
5. resize images to be the same dimensions/size, namely shape (224, 224) for this project
6. return the modified image

helpful resources: 
https://www.tensorflow.org/guide/data
https://www.tensorflow.org/tutorials/load_data/images
'''
def process_image(image_path, img_height=IMG_SIZE_HEIGHT, img_width=IMG_SIZE_WIDTH ):
    '''
    Takes an image file path and turns image into a Tensor
    '''

    # read image file (from TF load images documentation)
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)


    # Ensure square via padding before resize to avoid aspect distortion
    h = tf.shape(image)[0]
    w = tf.shape(image)[1]
    side = tf.maximum(h, w)
    image = tf.image.pad_to_bounding_box(image, (side - h)//2, (side - w)//2, side, side)
    

    
    # convert color channels from 0-255 to 0-1 values (Normalization)
    image = tf.image.convert_image_dtype(image, tf.float32)
    
    # resize image
    image = tf.image.resize(image, size=[img_height, img_width], method=tf.image.ResizeMethod.AREA)
    # return image converted to tensor
    return image


In [ ]:
# turining our data into batches
comment = '''
Why turn our data into batches?
Speeds up processing compared to trying to process all 10,000+ images in one go. All those images will probably not fit into RAM or VRAM (GPU)
Batch size of 32 is recommended. So process 32 images at a time

To use TF, data needs to be in the form of a Tensor tuples which look like:
(image, label)
'''

# function to return a tuple (image, label)

def get_image_label(image_path, label, img_height=IMG_SIZE_HEIGHT, img_width=IMG_SIZE_WIDTH):
    '''
    Takes image file path name and its label,
    processes the images and retunrs a tuple of (image, label)
    '''
    image = process_image(image_path, img_height=img_height, img_width=img_width)
    return image, label

In [ ]:
# turn all our data, X and y, into batches of size 32
comment = '''

'''

def create_data_batches(X, y=None, batch_size=BATCH_SIZE, shuffle=False):
    '''
    Creates batches of data out of image X and label y pairs
    shuffles the data if its training data (precautionary measure)
    doesn't shuffle if it's validation data
    Also accepts test data as input (no labels)
    '''

    if shuffle == False:
        # do not shuffle data
        if y is None:
            print('Creating data batches, no y labels provided...') # test data
            data = tf.data.Dataset.from_tensor_slices( tf.constant(X) ) # only filepaths, no labels
            data_batch = data.map(process_image).batch(batch_size)
        else: # if y labels provided
            print('Creating data batches, y labels provided...') # validation data
            data = tf.data.Dataset.from_tensor_slices( (tf.constant(X), tf.constant(y, dtype=tf.float32)) ) # filepaths, labels
            data_batch = data.map(get_image_label).batch(batch_size)
    else:
        # do shuffle data
        if y is None:
            print('Creating data batches, no y labels provided. Shuffling data...') # ? no need to shuffle non-training data that has no labels provided
            data = tf.data.Dataset.from_tensor_slices( tf.constant(X) ) # only filepaths, no labels
            # shuffle all the data
            data = data.shuffle(buffer_size=len(X))
            data_batch = data.map(process_image).batch(batch_size)
        else: # if y labels provided
            print('Creating data batches, y labels provided. Shuffling data...') # training data
            data = tf.data.Dataset.from_tensor_slices( (tf.constant(X), tf.constant(y, dtype=tf.float32)) ) # filepaths, labels
            # shuffle all the data
            data = data.shuffle(buffer_size=len(X)) 
            # create (image, label) tuples
            data_batch = data.map(get_image_label).batch(batch_size)

    return data_batch

In [ ]:
# create training and validation data batches
# shuffle training data just in case there is spurious structure in the ordering of the data that may adversely affect model training
train_data = create_data_batches(X_train, y_train, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# create validation data batches
val_data = create_data_batches(X_val, y_val, batch_size=BATCH_SIZE, shuffle=False) # no need to shuffle validation data

In [ ]:
# create test data batches
test_data = create_data_batches(X_test, y_test, batch_size=BATCH_SIZE, shuffle=False) # no need to shuffle test data

In [ ]:
# check attriubutes of our data batches
train_data.element_spec, val_data.element_spec, test_data.element_spec

In [ ]:
# pick a model from TensorFlow Hub that suits our problem (transfer learning)
comments = '''
Before we build a model, there are a few things we need to define
1. the input shape (in the form of tensors) to our model
2. the output shape (in the form of tensors) of our model
3. the URL of the model we want to use
'''

# setup input shape to the model
INPUT_SHAPE = [None, IMG_SIZE_HEIGHT, IMG_SIZE_WIDTH, 3] #[Batch, Height, Width, Color_Channels]

# output shape
OUTPUT_SHAPE = len(unique_labels) # an array of unique classification labels

# Setup model URL from TensorFlow Hub. See: https://www.kaggle.com/models/tensorflow/efficientnet/TensorFlow2/b5-feature-vector/1
MODEL_URL = 'https://www.kaggle.com/models/tensorflow/efficientnet/TensorFlow2/b5-feature-vector/1'

In [ ]:
# Put it all together into a keras deep-learning model!
# use the TF keras API. Keras is a user friendly high level API for building tensorflow models
comment = '''
create function that takes input of 
input shape
output shape
model

Function defines layers in sequential fashion, compiles model, builds model, returns the model

see: https://www.tensorflow.org/guide/keras/overview
'''

def create_model( input_shape = INPUT_SHAPE, output_shape = OUTPUT_SHAPE, model_url = MODEL_URL):
    print("Building model with: ".format(model_url))

    # setup model layers
    model = keras.Sequential([
            hub.KerasLayer(model_url), # Layer 1 (input layer)
            keras.layers.Dense(units = output_shape, activation="softmax"), # Layer 2 (output layer), for multi-class classficiation use softmax activation       
        ])

    # compile model
    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(), # for multi-class classficiation with integer labels use SparseCategoricalCrossentropy
        optimizer = keras.optimizers.Adam(),
        metrics=[
            "accuracy", # Fraction of correct predictions.
        ]
    )

    # build model
    model.build(input_shape)

    return model

In [ ]:
# Create model and display summary
model = create_model( input_shape = INPUT_SHAPE, output_shape = OUTPUT_SHAPE, model_url = MODEL_URL )
model.summary()

In [ ]:
# create callbacks: helper functions a model can use during training to save progress, check progress, or stop training early if model does not improve
# see: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard
comment = '''
To start a TensorBoard session from VSC:

1. Open the command palette (Ctrl/Cmd + Shift + P)
2. Search for the command “Python: Launch TensorBoard” and press enter.
3. You will be able to select the folder where your TensorBoard log files are located. 
   By default, the current working directory will be used.
'''




In [ ]:
# create function to build a tensorboard callback
def create_tensorboard_callback():
    logdir = os.path.join("logs/",datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
    return keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
# create function to stop training if it is not improving significantly anymore. Early stopping. See: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping

early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)


In [ ]:
# model checkpoint
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath= os.path.join("saved_models_tensorflow/",datetime.datetime.now().strftime('%Y%m%d-%H%M%S'),'best_model.keras'), 
    save_weights_only=False, # Set to True if you only want to save weights
    monitor='val_accuracy', # Metric to monitor (e.g., 'val_accuracy', 'val_loss')
    mode='max', # 'min' for loss, 'max' for accuracy
    save_best_only=True,
)

In [ ]:
# training a model on a subset of data (only 1000 images to make sure program works before training on 10,000+ images)
#NUM_EPOCHS = 10
NUM_EPOCHS = 50




In [ ]:
# Train and return a trained model

def train_model():
    '''
    Trains a model and returns the trained version
    '''
    # create model
    model = create_model()

    # create new tensorboard session everytime we train a model
    tensorboard_callback = create_tensorboard_callback()

    # fit model, also passing it the callbacks
    model.fit(
        x=train_data, 
        epochs=NUM_EPOCHS,
        validation_data = val_data,
        validation_freq = 1,
        callbacks = [tensorboard_callback, early_stopping_callback, model_checkpoint_callback],
    )

    # return fitted model
    return model

In [ ]:
# Fit model to data
model = train_model()

In [ ]:
# Load Best Model
search_pattern = 'saved_models_tensorflow/**/best_model.keras'
list_of_files = glob.glob(search_pattern, recursive=True)

if not list_of_files:
    print("No file named 'best_model.keras' found in 'saved_models_tensorflow' subfolders.")
else:
    # Find the file with the maximum creation time (os.path.getctime)
    # or modification time (os.path.getmtime)
    latest_file = max(list_of_files, key=os.path.getctime)
    print(f"The latest created file is: {latest_file}")


In [ ]:
# try loading the best model
# note, need to add custom_objects={"KerasLayer":hub.KerasLayer} because of using TensorHub
loaded_best_model = keras.models.load_model(latest_file, custom_objects={"KerasLayer":hub.KerasLayer})

In [ ]:
# View TensorBoard


In [ ]:
# Evaluation metrics on Train, Validation, and Test data
metric_names = ["Loss", "Accuracy"]

In [ ]:
# -----------------------
# Evaluate Train Metrics
# -----------------------
train_metrics = loaded_best_model.evaluate(train_data, verbose=0);

train_metrics_df = pd.DataFrame({
    "Metric_Name": metric_names,
    "Value": train_metrics,
})

train_metrics_df

In [ ]:
# -----------------------
# Evaluate Validation Metrics
# -----------------------

val_metrics = loaded_best_model.evaluate(val_data, verbose=0);

val_metrics_df = pd.DataFrame({
    "Metric_Name": metric_names,
    "Value": val_metrics,
})

val_metrics_df

In [ ]:
# -----------------------
# Evaluate Test Metrics
# -----------------------

test_metrics = loaded_best_model.evaluate(test_data, verbose=0);

test_metrics_df = pd.DataFrame({
    "Metric_Name": metric_names,
    "Value": test_metrics,
})

test_metrics_df

In [ ]:
# Now predict at scale with visualization of images

# turn probabilities into their respective label (for ease of understanding)

def get_pred_label(prediction_probs, class_names):
    """
    Convert model prediction probabilities to class label.
    """
    class_idx = int(np.argmax(prediction_probs))
    return class_names[class_idx]


In [ ]:
# predictions

train_data_predictions = loaded_best_model.predict(train_data, verbose=1)
val_data_predictions = loaded_best_model.predict(val_data, verbose=1)
test_data_predictions = loaded_best_model.predict(test_data, verbose=1)

In [ ]:
# need to unbatch the train_data to get the predictions, images and true labels
train_data_images_unbatched_list = []
train_data_labels_unbatched_list = []

# loop through unbatched data
for image, label in train_data.unbatch().as_numpy_iterator():
    train_data_images_unbatched_list.append(image)
    train_data_labels_unbatched_list.append(label)

In [ ]:
# need to unbatch the val_data to get the predictions, images and true labels
val_data_images_unbatched_list = []
val_data_labels_unbatched_list = []

# loop through unbatched data
for image, label in val_data.unbatch().as_numpy_iterator():
    val_data_images_unbatched_list.append(image)
    val_data_labels_unbatched_list.append(label)

In [ ]:
# need to unbatch the test_data to get the predictions, images and true labels
test_data_images_unbatched_list = []
test_data_labels_unbatched_list = []

# loop through unbatched data
for image, label in test_data.unbatch().as_numpy_iterator():
    test_data_images_unbatched_list.append(image)
    test_data_labels_unbatched_list.append(label)

In [ ]:
train_data_unbatched_label = [get_pred_label(label) for label in train_data_labels_unbatched_list] # true label of validation data
train_predictions_label = [get_pred_label(label) for label in train_data_predictions] # prediction on validation data

In [ ]:
val_data_unbatched_label = [get_pred_label(label) for label in val_data_labels_unbatched_list] # true label of validation data
val_predictions_label = [get_pred_label(label) for label in val_data_predictions] # prediction on validation data

In [ ]:
test_data_unbatched_label = [get_pred_label(label) for label in test_data_labels_unbatched_list] # true label of validation data
test_predictions_label = [get_pred_label(label) for label in test_data_predictions] # prediction on validation data

In [ ]:
# create dataframes

Train_Results_df = pd.DataFrame(
    {
        'True_Label': train_data_unbatched_label,
        'Prediction_Label': train_predictions_label,
        'Image_Data': train_data_images_unbatched_list,
    }
)


Validation_Results_df = pd.DataFrame(
    {
        'True_Label': val_data_unbatched_label,
        'Prediction_Label': val_predictions_label,
        'Image_Data': val_data_images_unbatched_list,
    }
)

Test_Results_df = pd.DataFrame(
    {
        'True_Label': test_data_unbatched_label,
        'Prediction_Label': test_predictions_label,
        'Image_Data': test_data_images_unbatched_list,
    }
)

In [ ]:
# show some rows of dataframe
Train_Results_df[['True_Label', 'Prediction_Label']]

In [ ]:
# show some rows of dataframe
Validation_Results_df[['True_Label', 'Prediction_Label']]

In [ ]:
# show some rows of dataframe
Test_Results_df[['True_Label', 'Prediction_Label']]

In [ ]:
# Training Confusion Matrix

train_cm = confusion_matrix(Train_Results_df['True_Label'], Train_Results_df['Prediction_Label'])

disp = ConfusionMatrixDisplay(confusion_matrix=train_cm, display_labels=unique_labels)

disp.plot(xticks_rotation=60)



In [ ]:
# Validation Confusion Matrix

val_cm = confusion_matrix(Validation_Results_df['True_Label'], Validation_Results_df['Prediction_Label'])

disp = ConfusionMatrixDisplay(confusion_matrix=val_cm, display_labels=unique_labels)

disp.plot(xticks_rotation=60)

In [ ]:
# Test Confusion Matrix

test_cm = confusion_matrix(Test_Results_df['True_Label'], Test_Results_df['Prediction_Label'])

disp = ConfusionMatrixDisplay(confusion_matrix=test_cm, display_labels=unique_labels)

disp.plot(xticks_rotation=60)

In [ ]:
# Train Classification Report

train_report = classification_report(Train_Results_df['True_Label'], Train_Results_df['Prediction_Label'])
print(train_report)

In [ ]:
# Validation Classification Report

val_report = classification_report(Validation_Results_df['True_Label'], Validation_Results_df['Prediction_Label'])
print(val_report)

In [ ]:
# Test Classification Report

test_report = classification_report(Test_Results_df['True_Label'], Test_Results_df['Prediction_Label'])
print(test_report)

In [ ]:

# # Figures Test Results (Plotly)

# num_rows = np.ceil(np.sqrt(len(Test_Results_df))).astype(int)
# num_cols = num_rows

# df = Test_Results_df.head(100)  # shorthand, first 100 rows
# n = len(df)
# grid = int(np.ceil(np.sqrt(n)))

# # Create list of subplot titles
# titles = [
#     f"True Label: {row['True_Label']}<br>Predicted: {row['Prediction_Label']}"
#     for _, row in df.iterrows()
# ]

# # create figure
# fig = make_subplots(rows=grid, cols=grid, subplot_titles=titles, vertical_spacing=0.02, horizontal_spacing=0.01)



# def to_uint8(img):
#     """Ensure image is uint8 RGB for Plotly."""
#     arr = np.asarray(img)
#     if arr.dtype == np.uint8:
#         return arr
#     # If floats in [0,1], scale up; otherwise clip to [0,255]
#     if np.issubdtype(arr.dtype, np.floating) and arr.max() <= 1.0:
#         arr = (np.clip(arr, 0.0, 1.0) * 255).astype(np.uint8)
#     else:
#         arr = np.clip(arr, 0, 255).astype(np.uint8)
#     return arr

# for idx, img in enumerate(df["Image_Data"]):
#     r = idx // grid + 1
#     c = idx % grid + 1
#     img_u8 = to_uint8(img)            # expects shape (H, W, 3)
#     fig.add_trace(go.Image(z=img_u8), row=r, col=c)

# # Hide ticks/axes and keep a nice square layout
# fig.update_xaxes(visible=False)
# fig.update_yaxes(visible=False)

# # Optional: make figure size scale with grid
# fig.update_layout(
#     showlegend=False,
#     #margin=dict(l=10, r=10, t=10, b=10),
#     height=grid * 200,
#     width=grid * 200,
#     font=dict(size=2)
# )

# # Update the font size of all annotations (subplot titles)
# fig.update_annotations(font_size=12) # Change 12 to your desired font size

# fig.show()



In [ ]:
# Figure using Matplotlib

import matplotlib.pyplot as plt

# --- Helper to ensure uint8 ---
def to_uint8(img):
    arr = np.asarray(img)
    if arr.dtype == np.uint8:
        return arr
    if np.issubdtype(arr.dtype, np.floating) and arr.max() <= 1.0:
        arr = (np.clip(arr, 0.0, 1.0) * 255).astype(np.uint8)
    else:
        arr = np.clip(arr, 0, 255).astype(np.uint8)
    return arr

# --- Setup Data ---
df = Test_Results_df.head(100)
n = len(df)
grid = int(np.ceil(np.sqrt(n)))

# --- Create Matplotlib Figure ---
fig, axes = plt.subplots(grid, grid, figsize=(grid * 2, grid * 2))
axes = axes.flatten()   # flatten for easy indexing

for idx, (i, row) in enumerate(df.iterrows()):
    ax = axes[idx]
    img = to_uint8(row["Image_Data"])
    ax.imshow(img)
    ax.set_title(
        f"True: {row['True_Label']}\nPred: {row['Prediction_Label']}",
        fontsize=8
    )
    ax.axis("off")

# Turn off any extra empty subplots
for j in range(idx + 1, len(axes)):
    axes[j].axis("off")

plt.tight_layout()
plt.show()


# Test-Set Performance Evaluation

Your model is a 4-class image classifier for **cataracts**, **diabetic retinopathy**, **glaucoma**, and **normal** retinal images.  
Below is a structured interpretation of your test metrics.

---

## Overall Metrics
- **Loss:** `0.3305`  
- **Accuracy:** `0.8873`  
- **Macro F1 Score:** `0.8840`

**Interpretation:**  
These are strong results for a 4-class medical image classification task. Accuracy near 0.89 is well above the 25% random baseline, and an F1 around 0.88 indicates balanced and reliable predictions across classes.

---

## Per-Class Precision

| Class | Precision | Notes |
|-------|-----------|-------|
| **Cataract** | `0.9102` | Very strong; few false positives. |
| **Diabetic Retinopathy** | `0.9727` | Excellent; highly reliable predictions. |
| **Glaucoma** | `0.8161` | Weaker; more false positives. |
| **Normal** | `0.8865` | Very good performance. |

**Summary:**  
Precision is strong overall, especially for diabetic retinopathy. Glaucoma is the only class showing moderate precision.

---

## Per-Class Recall

| Class | Recall | Notes |
|-------|--------|-------|
| **Cataract** | `0.8889` | High sensitivity. |
| **Diabetic Retinopathy** | `0.9596` | Outstanding recall. |
| **Glaucoma** | `0.8023` | Lowest recall; misses some glaucoma cases. |
| **Normal** | `0.8286` | Good, but could be improved. |

**Summary:**  
Recall is consistently high, but glaucoma remains the most challenging class.

---

## High-Level Assessment
- **Performance is robust and clinically meaningful.**
- The model reliably identifies all classes, especially diabetic retinopathy.
- **Glaucoma** shows the weakest performance (expected due to subtle features).
- No sign of extreme class imbalance or overfitting based on metrics alone.


---

## Summary
The model demonstrates **strong overall performance** with **Accuracy ~88.7%** and **Macro F1 ~88.4%**, delivering reliable detection across all four eye-condition classes.  
Minor weaknesses in glaucoma detection are expected and can be improved with targeted techniques.

---


In [ ]:
# Write a csv file of test_df with predictions
timestamp = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
test_results_output_df = test_df.copy()
test_results_output_df['Predicted_Label'] = Test_Results_df['Prediction_Label'].values.tolist()
output_csv_path = f"test_results_with_predictions_tensorflow_{timestamp}.csv"
test_results_output_df.to_csv(output_csv_path, index=False)
print(f"Test results with predictions saved to {output_csv_path}")